In [ ]:
Q1

In [ ]:
SELECT L_RETURNFLAG, L_LINESTATUS, SUM(L_QUANTITY) AS SUM_QTY,
SUM(L_EXTENDEDPRICE) AS SUM_BASE_PRICE,
SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT)) AS SUM_DISC_PRICE,
SUM(dbo.discount_taxprice(L_EXTENDEDPRICE, L_DISCOUNT, L_TAX)) AS SUM_CHARGE,
AVG(L_QUANTITY) AS AVG_QTY,
AVG(L_EXTENDEDPRICE) AS AVG_PRICE, AVG(L_DISCOUNT) AS AVG_DISC,
COUNT(*) AS COUNT_ORDER
FROM LINEITEM
WHERE dbo.isShippedBefore(L_SHIPDATE, -90, '1998-12-01') = 1
GROUP BY L_RETURNFLAG, L_LINESTATUS
ORDER BY L_RETURNFLAG, L_LINESTATUS;

In [ ]:
l_returnflag | l_linestatus |    sum_qty    |    sum_base_price    |    sum_disc_price    |    sum_charge    |   avg_qty   |   avg_price   |   avg_disc   |  count_order
-------------+-------------+--------------+---------------------+---------------------+-----------------+-------------+--------------+-------------+--------------
A            | F           | 37734107.00   | 566865540.73         | 53758257382.21      | 55909066592.19  | 25.522005   | 38273.129734 | 0.049985    | 1478493
N            | F           | 991417.00     | 1487504710.38        | 1413082173.74       | 14696449232.14  | 25.516471   | 38284.467760 | 0.050093    | 38854
N            | O           | 74476040.00   | 111701729697.74      | 106118230774.99     | 110367044585.55 | 25.502226   | 38249.117988 | 0.049996    | 2920374
R            | F           | 37719753.00   | 56568041380.90       | 53741292921.05      | 55589619477.81  | 25.505793   | 38250.854626 | 0.050009    | 1478870
(4 行记录)

In [ ]:
Q3

In [ ]:
SELECT TOP 10 L_ORDERKEY,
SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE,
O_ORDERDATE, O_SHIPPRIORITY
FROM CUSTOMER, ORDERS, LINEITEM
WHERE C_CUSTKEY = O_CUSTKEY AND L_ORDERKEY = O_ORDERKEY
AND dbo.q3conditions(C_MKTSEGMENT, O_ORDERDATE, L_SHIPDATE) = 1
GROUP BY L_ORDERKEY, O_ORDERDATE, O_SHIPPRIORITY
ORDER BY REVENUE DESC, O_ORDERDATE;

In [ ]:
l_orderkey |   revenue   | o_orderdate | o_shippriority
-----------+-------------+-------------+---------------
2456423   | 406181.00   | 1995-03-05  | 0
3459600   | 405638.70   | 1995-03-04  | 0
492164    | 390324.06   | 1995-02-19  | 0
1186320   | 364537.93   | 1995-03-09  | 0
2435712   | 378673.06   | 1995-02-26  | 0
4878020   | 378376.79   | 1995-03-12  | 0
5521732   | 375153.93   | 1995-03-13  | 0
2628192   | 373133.31   | 1995-02-22  | 0
933600    | 371407.45   | 1995-03-05  | 0
2300070   | 367371.14   | 1995-03-13  | 0
(10 行记录)

In [ ]:
Q5

In [ ]:
SELECT N_NAME,
SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE
FROM CUSTOMER, ORDERS, LINEITEM, SUPPLIER, NATION, REGION
WHERE C_CUSTKEY = O_CUSTKEY AND L_ORDERKEY = O_ORDERKEY
AND L_SUPPKEY = S_SUPPKEY AND C_NATIONKEY = S_NATIONKEY
AND S_NATIONKEY = N_NATIONKEY AND N_REGIONKEY = R_REGIONKEY
AND dbo.q5Conditions(R_NAME, O_ORDERDATE) = 1
GROUP BY N_NAME
ORDER BY REVENUE DESC;

In [ ]:
n_name      |       revenue
-----------+-------------------
INDONESIA  | 55502041.37
VIETNAM    | 55295087.24
CHINA      | 53724494.43
INDIA      | 52035512.19
JAPAN      | 45410175.92
(5 行记录)

In [ ]:
Q6

In [ ]:
SELECT SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE
FROM LINEITEM
WHERE dbo.q6conditions(L_SHIPDATE, L_DISCOUNT, L_QUANTITY) = 1;

In [ ]:
REVENUE
-------------------+
123141078.2283
(1 行记录)

In [ ]:
Q7

In [ ]:
SELECT SUPP_NATION, CUST_NATION, L_YEAR, SUM(VOLUME) AS REVENUE
FROM (
    SELECT N1.N_NAME AS SUPP_NATION, N2.N_NAME AS CUST_NATION,
    datepart(yy, L_SHIPDATE) AS L_YEAR,
    L_EXTENDEDPRICE * (1 - L_DISCOUNT) AS VOLUME
    FROM SUPPLIER, LINEITEM, ORDERS, CUSTOMER, NATION N1, NATION N2
    WHERE S_SUPPKEY = L_SUPPKEY AND O_ORDERKEY = L_ORDERKEY
    AND C_CUSTKEY = O_CUSTKEY
    AND S_NATIONKEY = N1.N_NATIONKEY
    AND C_NATIONKEY = N2.N_NATIONKEY
    AND dbo.q7conditions(N1.N_NAME, N2.N_NAME, L_SHIPDATE) = 1
) AS SHIPPING
GROUP BY SUPP_NATION, CUST_NATION, L_YEAR
ORDER BY SUPP_NATION, CUST_NATION, L_YEAR;

In [ ]:
supp_nation | cust_nation | l_year |    revenue
-----------+------------+--------+---------------
FRANCE     | GERMANY    | 1995   | 54639732.7336
FRANCE     | GERMANY    | 1996   | 54633083.3076
GERMANY    | FRANCE     | 1995   | 52531746.6697
GERMANY    | FRANCE     | 1996   | 52520549.0224
(4 行记录)

In [ ]:
Q9

In [ ]:
SELECT NATION, O_YEAR, SUM(AMOUNT) AS SUM_PROFIT
FROM (
    SELECT N_NAME AS NATION,
    datepart(yy, O_ORDERDATE) AS O_YEAR,
    dbo.profit_amount(L_EXTENDEDPRICE, L_DISCOUNT, PS_SUPPLYCOST, L_QUANTITY) AS AMOUNT
    FROM PART, SUPPLIER, LINEITEM, PARTSUPP, ORDERS, NATION
    WHERE S_SUPPKEY = L_SUPPKEY AND PS_SUPPKEY = L_SUPPKEY
    AND PS_PARTKEY = L_PARTKEY AND P_PARTKEY = L_PARTKEY
    AND O_ORDERKEY = L_ORDERKEY AND S_NATIONKEY = N_NATIONKEY AND
    P_NAME LIKE '%green%'
) AS PROFIT
GROUP BY NATION, O_YEAR
ORDER BY NATION, O_YEAR DESC;

In [ ]:
nation       | o_year |      sum_profit
------------+-------+-------------------
ALGERIA      | 1995  | 27136900.41
ALGERIA      | 1997  | 48611833.70
ALGERIA      | 1996  | 48285482.91
ALGERIA      | 1995  | 44402273.96
ALGERIA      | 1994  | 48694008.22
ALGERIA      | 1993  | 46044208.08
ALGERIA      | 1992  | 45636849.67
ARGENTINA    | 1996  | 28341663.81
ARGENTINA    | 1997  | 47143964.50
ARGENTINA    | 1996  | 45255278.82
(10 行记录)

In [ ]:
Q10

In [ ]:
SELECT TOP 20 C_CUSTKEY, C_NAME,
SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE,
C_ACCTBAL, N_NAME, C_ADDRESS, C_PHONE, C_COMMENT
FROM CUSTOMER, ORDERS, LINEITEM, NATION
WHERE C_CUSTKEY = O_CUSTKEY AND L_ORDERKEY = O_ORDERKEY AND
dbo.q10conditions(O_ORDERDATE, L_RETURNFLAG) = 1
AND C_NATIONKEY = N_NATIONKEY
GROUP BY C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE,
N_NAME, C_ADDRESS, C_COMMENT
ORDER BY REVENUE DESC;

In [ ]:
c_custkey |          c_name          |  revenue  |  c_acctbal  | n_name |                     c_address                      |          c_phone          |                                                                 c_comment
---------+-------------------------+----------+------------+--------+---------------------------------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------
57040    | Customer#000057040   | 734235.24   | 2455       | JAPAN   | Eioyzj4pp                                           | 22-895-641-3466      | sits. slyly regular requests sleep alongside of the regular inst
143347   | Customer#000143347   | 721002.68  | 9648       | EGYPT  | 1a8eFYv, Kw4                                        | 14-742-935-3718      | ggle carefully enticing requests. final deposits use bold, bold pinto beans. ironic, idle re
60838    | Customer#000060838   | 679127.31  | 3077       | BRAZIL | 64fajTsWHAWJLbQxJk1pNc2RTjWE                        | 12-913-494-9813      | need to boost against the slyly regular account
101998   | Customer#000101998   | 637029.56  | 5667       | UNITED KINGDOM | 01o9CILnNtF00YmZj                                | 33-593-665-6378      | ress foxes wake slyly after the bold excuses. ironic platlets are furiously carefully bold theodolites
125341   | Customer#000125341   | 633508.08  | 8060       | GERMANY | S29000b6oeU89SuEfJnkWaK                            | 17-582-695-5962      | arefully even depths. blithely even excuses sleep furiously. foxes use except the dependencies. oa
25501    | Customer#000025501   | 620269.79  | 7849       | ETHIOPIA | W56MXuoi aYCCZamJL RoD8AJCIGdQ8DIZ                 | 15-874-608-6793      | he pending instructions wake carefully at the pinto beans. regular, final instructions doze the slyly fina
115831   | Customer#000115831   | 596423.86  | 8672       | FRANCE  | rFeBEyk dI ne7z5FDmigl0K09wYrXgCgIo               | 16-715-386-3788      | l somas sleep. furiously final deposits wake blithely regular pinto b
84223    | Customer#000084223   | 594998.02  | 2339       | UNITED KINGDOM | naVZCs6BaWap rrM2ZN 2qhns6WbaunbA                | 33-442-824-8191      | slyly final deposits hassle regular, pending dependencies. pending escapes wake bl
54289    | Customer#000054289   | 586603.39  | 3918       | IRAN    | vXcx0c5U0Ba45jQT , oobkZ                          | 20-834-292-4707      | ely special foxes are quickly finally ironic p
399222    | Customer#000039922   | 564878.11  | 11343      | GERMANY | Zg2v4s5OI2GKM4pLDPBUm342zWer                      | 17-147-757-6036      | y final requests. furiously final foxes cajole blithely special platlets. f
(10 行记录)

In [ ]:
Q11

In [ ]:
SELECT PS_PARTKEY, SUM(PS_SUPPLYCOST * PS_AVAILQTY) AS VALUE
FROM PARTSUPP, SUPPLIER, NATION
WHERE PS_SUPPKEY = S_SUPPKEY AND S_NATIONKEY = N_NATIONKEY
AND N_NAME = 'GERMANY'
GROUP BY PS_PARTKEY
HAVING SUM(PS_SUPPLYCOST * PS_AVAILQTY) > dbo.total_value()
ORDER BY VALUE DESC;

In [ ]:
ps_partkey |       value
-----------+---------------
129760     | 17538456.86
166726     | 18503353.92
191287     | 16474001.97
161758     | 16101755.54
34452      | 15963844.72
139035     | 15907078.34
9403       | 15451755.62
154358     | 15212937.88
38823      | 15064802.86
85606      | 15053957.15
(10 行记录)

In [ ]:
Q12

In [ ]:
SELECT L_SHIPMODE,
SUM(dbo.line_count(O_ORDERPRIORITY, 'high')) AS HIGH_LINE_COUNT,
SUM(dbo.line_count(O_ORDERPRIORITY, 'low')) AS LOW_LINE_COUNT
FROM ORDERS, LINEITEM
WHERE O_ORDERKEY = L_ORDERKEY AND
dbo.q12conditions(L_SHIPMODE, L_COMMITDATE, L_RECEIPTDATE, L_SHIPDATE) = 1
GROUP BY L_SHIPMODE
ORDER BY L_SHIPMODE;

In [ ]:
l_shipmode | high_line_count | low_line_count
-----------+----------------+---------------
MAIL       | 10746         | 10746
SHIP       | 10864         | 10864
(2 行记录)

In [ ]:
Q14

In [ ]:
SELECT 100.00 *
SUM(dbo.promo_disc(P_TYPE, L_EXTENDEDPRICE, L_DISCOUNT))
/ SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT))
AS PROMO_REVENUE
FROM LINEITEM, PART
WHERE L_PARTKEY = P_PARTKEY AND L_SHIPDATE >= '1995-09-01'
AND L_SHIPDATE < dateadd(mm, 1, '1995-09-01');

In [ ]:
PROMO_REVENUE
-------------------+
16.380778
(1 行记录)

In [ ]:
Q19

In [ ]:
SELECT SUM(dbo.discount_price(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE
FROM LINEITEM join PART on L_PARTKEY = P_PARTKEY
WHERE dbo.q19conditions(P_CONTAINER, L_QUANTITY, P_SIZE,
L_SHIPMODE, L_SHIPINSTRUCT, P_BRAND) = 1;

In [ ]:
REVENUE
-------------------+
3083643.03
(1 行记录)

In [ ]:
Q22

In [ ]:
SELECT CNTRYCODE,
COUNT(*) AS NUMCUST, SUM(C_ACCTBAL) AS TOTACCTBAL
FROM (
    SELECT SUBSTRING(C_PHONE, 1, 2) AS CNTRYCODE, C_ACCTBAL
    FROM CUSTOMER WHERE SUBSTRING(C_PHONE, 1, 2)
    IN ('13', '31', '23', '29', '30', '18', '17')
    AND C_ACCTBAL > dbo.avg_actbal()
    AND NOT EXISTS (SELECT * FROM ORDERS WHERE O_CUSTKEY = C_CUSTKEY)
) AS CUSTSALE
GROUP BY CNTRYCODE
ORDER BY CNTRYCODE;

In [ ]:
entrycode | numcust |  totacctbal
---------+---------+------------
    13   |   888   | 6737713.99
    17   |   861   | 6400573.72
    18   |   964   | 7236687.40
    23   |   892   | 6701457.95
    29   |   948   | 7158866.63
    30   |   909   | 6808436.13
    31   |   922   | 6806670.18
(7 行记录)